Important resources:

DEMOGRAPHICS DATASET

https://www.census.gov/data/tables/time-series/demo/popest/2020s-counties-detail.html
scroll down to "Annual County Resident Population Estimates by Age, Sex, Race, and Hispanic Origin: April 1, 2020 to July 1, 2022 (CC-EST2022-ALLDATA)"
and click "United States" to download csv file for county demographics (name of file should be cc-est2022-all.csv)

https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2022/cc-est2022-alldata.pdf this shows what each column name means in terms of demographics

NRI DATASET

https://hazards.fema.gov/nri/data-resources#csvDownload
scroll down to "Table Format (CSV)" and click "All Counties - Country-level detail (Table)" and name of folder should be NRI_Table_Counties. the csv file has the same name  within the folder (NRI_Table_Counties.csv)


In [4]:
import pandas as pd

In [7]:
#read national risk index dataset into the nri dataframe
nri = pd.read_csv("NRI_Table_Counties/NRI_Table_Counties.csv")

#only include important columns
nri = nri[["NRI_ID", "STATE", "COUNTY", "POPULATION", "RISK_VALUE", "RISK_SCORE", "RISK_RATNG"]]

#nri has some extra territories, so let's define it in a list...
territories = ["American Samoa", "Guam", "Northern Mariana Islands", "Puerto Rico", "Virgin Islands"]
#...and get rid of the rows with these territories using .drop()
nri = nri.drop(nri[nri["STATE"].isin(territories)].index)

#now we should only have the 50 states + DC
nri


,NRI_ID,STATE,COUNTY,POPULATION,RISK_VALUE,RISK_SCORE,RISK_RATNG
0,C01001,Alabama,Autauga,58764,6.156054e+06,49.220490,Relatively Low
1,C01003,Alabama,Baldwin,231365,2.106327e+08,97.709195,Relatively High
2,C01005,Alabama,Barbour,25160,7.412840e+06,56.188355,Relatively Low
3,C01007,Alabama,Bibb,22239,3.863747e+06,32.484887,Very Low
4,C01009,Alabama,Blount,58992,1.023854e+07,65.128858,Relatively Low
...,...,...,...,...,...,...,...
3138,C56037,Wyoming,Sweetwater,42238,1.189081e+06,6.617881,Very Low
3139,C56039,Wyoming,Teton,23250,1.141156e+07,68.501432,Relatively Low
3140,C56041,Wyoming,Uinta,20412,2.821381e+06,21.699014,Very Low
3141,C56043,Wyoming,Washakie,7662,6.211963e+05,2.513522,Very Low


In [9]:
#read demographics dataset into the demo dataframe
#the encoding part is because python reads in a specific encoding of the file, but file isn't in that form, so this fixes that
demo = pd.read_csv("cc-est2022-all.csv", encoding='iso-8859-1').dropna()

#making demo only include year 4, which is population estimate from 7/1/2022
demo = demo[(demo.YEAR == 4)]

#each county also had "County" at the end, so let's remove that and put them in a new column that is clearer
demo['COUNTY'] = demo['CTYNAME'].str.replace(" County", "")

#making new column for states bc "stname" is stupid
demo["STATE"] = demo["STNAME"]

#context: each county has several rows, 1 for each age-group
#using .groupby() to combine all the age groups so each county takes up 1 row
#using sort=False to keep them in the same orders
demo = demo.groupby(["COUNTY", "STATE"], sort=False).agg("sum").reset_index()

#removing the columns we don't need
demo = demo.drop(["STNAME","CTYNAME","SUMLEV","AGEGRP"], axis=1)
demo


,COUNTY,STATE,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,Autauga,Alabama,76,119518,58210,61308,44052,45342,11912,13606,...,1804,1620,264,224,94,80,46,44,44,38
1,Baldwin,Alabama,76,492870,240328,252542,210450,220288,20202,21316,...,11714,10962,740,840,706,654,236,264,176,172
2,Barbour,Alabama,76,49412,26128,23284,12846,11452,12434,11144,...,1098,884,190,126,144,70,20,18,66,36
3,Bibb,Alabama,76,44010,23550,20460,17274,16400,5728,3518,...,634,512,80,64,36,42,16,20,38,12
4,Blount,Alabama,76,119024,59166,59858,56300,57236,1280,1030,...,6056,5304,208,160,160,130,30,58,74,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,Sweetwater,Wyoming,76,82690,42938,39752,40272,37364,646,448,...,6804,5984,192,154,432,398,70,86,44,44
3140,Teton,Wyoming,76,46574,24412,22162,23040,20832,220,172,...,3590,3240,54,60,248,218,42,44,26,26
3141,Uinta,Wyoming,76,41424,21196,20228,20240,19240,146,126,...,1896,1840,54,46,196,244,14,32,12,14
3142,Washakie,Wyoming,76,15438,7946,7492,7510,7068,52,32,...,1014,966,22,20,116,96,12,16,12,4


In [10]:
#displaying the rows in nri that are in connecticut
nri[nri.STATE == "Connecticut"]
#these need to get FIXED

,NRI_ID,STATE,COUNTY,POPULATION,RISK_VALUE,RISK_SCORE,RISK_RATNG
309,C09001,Connecticut,Fairfield,957322,1.084730e+08,95.450207,Relatively High
310,C09003,Connecticut,Hartford,898547,5.277812e+07,91.473115,Relatively Moderate
311,C09005,Connecticut,Litchfield,185062,2.323207e+07,82.500795,Relatively Low
312,C09007,Connecticut,Middlesex,164179,1.843823e+07,78.364620,Relatively Low
313,C09009,Connecticut,New Haven,864495,8.602981e+07,94.463888,Relatively Moderate
314,C09011,Connecticut,New London,268434,3.323352e+07,87.368756,Relatively Moderate
315,C09013,Connecticut,Tolland,149768,1.089484e+07,67.069679,Relatively Low
316,C09015,Connecticut,Windham,116384,1.178378e+07,69.519567,Relatively Low


In [12]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [13]:
#displaying the rows in nri that are in connecticut
demo[demo.STATE == "Connecticut"]

#main problem: when connecticut in demo has 8 counties, and 9 planning regions in nri
#this means the number of rows connecticut has in each dataframe isn't the same so we need to fix that

,COUNTY,STATE,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
309,Capitol Planning Region,Connecticut,76,1962894,958518,1004376,717904,751916,143094,154612,...,145496,151496,30172,33394,6944,7292,2776,2510,1772,1216
310,Greater Bridgeport Planning Region,Connecticut,76,654572,320890,333682,248442,254362,44642,49930,...,60664,60016,8760,9184,2666,2416,972,874,444,412
311,Lower Connecticut River Valley Planning Region,Connecticut,76,353244,173998,179246,153004,157306,10282,10470,...,10870,11470,1954,2108,516,558,220,226,106,102
312,Naugatuck Valley Planning Region,Connecticut,76,908166,443490,464676,342928,356634,65120,70650,...,77014,77646,14406,15042,3642,3576,1100,996,516,514
313,Northeastern Connecticut Planning Region,Connecticut,76,192392,96294,96098,87910,88350,3250,2648,...,9688,9054,1492,1468,630,614,254,170,144,94
314,Northwest Hills Planning Region,Connecticut,76,226468,112470,113998,97924,99418,7586,7522,...,10972,11130,2048,2164,616,588,208,182,102,104
315,South Central Connecticut Planning Region,Connecticut,76,1146488,553704,592784,424880,450198,82598,92734,...,92058,94924,16700,18046,4576,4666,1310,1226,668,610
316,Southeastern Connecticut Planning Region,Connecticut,76,560806,280286,280520,233018,235464,20670,18108,...,27356,26774,6350,6044,2386,2542,842,764,414,276
317,Western Connecticut Planning Region,Connecticut,76,1247380,612858,634522,487314,496002,70538,79868,...,113324,113280,14404,15892,4932,4674,1788,1642,838,734


In [14]:
#editing each CT county in nri to match planning regions and the associated counties

#connecticut: county | planning region associated (figured out by matching their locations)
#litchfield | northwestern
#hartford + tolland | capitol
#windham | northeastern
#fairfield | western + greater bridgeport
#new haven | naugatuck + south central
#middlesex | lower connecticut
#new london | southeastern

#for planning regions that had more than 1 county associated with it, the values for those counties were averages and put into the planning region

bridgeport = nri.loc[30].RISK_VALUE #used this to format everything in nri to be in scientific notation with the +e
nri.loc[309] = {"STATE":"Connecticut","COUNTY":"Capitol Planning Region", "RISK_VALUE":(5.277812e+07 + 1.089484e+07)/2, "RISK_SCORE": (91.473115+67.069679)/2, "RISK_RATNG":"Relatively Low"}
nri.loc[310] = {"STATE":"Connecticut","COUNTY":"Greater Bridgeport Planning Region", "RISK_VALUE":bridgeport, "RISK_SCORE": 95.450207	, "RISK_RATNG":"Relatively High"}
nri.loc[311] = {"STATE":"Connecticut","COUNTY":"Lower Connecticut River Valley Planning Region", "RISK_VALUE":1.843823e+07, "RISK_SCORE": 78.364620, "RISK_RATNG":"Relatively Low"}
nri.loc[312] = {"STATE":"Connecticut","COUNTY":"Naugatuck Valley Planning Region", "RISK_VALUE":8.602981e+07, "RISK_SCORE": 94.463888, "RISK_RATNG":"Relatively Moderate"}
nri.loc[313] = {"STATE":"Connecticut","COUNTY":"Northeastern Connecticut Planning Region", "RISK_VALUE":1.178378e+07, "RISK_SCORE":69.519567, "RISK_RATNG":"Relatively Low"}
nri.loc[314] = {"STATE":"Connecticut","COUNTY":"Northwest Hills Planning Region", "RISK_VALUE":2.323207e+07, "RISK_SCORE": 82.500795, "RISK_RATNG":"Relatively Low"}
nri.loc[315] = {"STATE":"Connecticut","COUNTY":"South Central Connecticut Planning Region", "RISK_VALUE":8.602981e+07, "RISK_SCORE": 94.463888, "RISK_RATNG":"Relatively Moderate"}
nri.loc[316] = {"STATE":"Connecticut","COUNTY":"Southeastern Connecticut Planning Region", "RISK_VALUE":3.323352e+07, "RISK_SCORE": 87.368756, "RISK_RATNG":"Relatively Moderate"}

#adding new row in nri to be the western planning region
nri.loc[316.5] = {"STATE":"Connecticut","COUNTY":"Western Planning Region", "RISK_VALUE":bridgeport, "RISK_SCORE": 95.450207	, "RISK_RATNG":"Relatively High"}
nri = nri.sort_index().reset_index(drop=True)

nri[nri.STATE == "Connecticut"] #all good


,NRI_ID,STATE,COUNTY,POPULATION,RISK_VALUE,RISK_SCORE,RISK_RATNG
309,NaN,Connecticut,Capitol Planning Region,NaN,3.183648e+07,79.271397,Relatively Low
310,NaN,Connecticut,Greater Bridgeport Planning Region,NaN,1.045320e+07,95.450207,Relatively High
311,NaN,Connecticut,Lower Connecticut River Valley Planning Region,NaN,1.843823e+07,78.364620,Relatively Low
312,NaN,Connecticut,Naugatuck Valley Planning Region,NaN,8.602981e+07,94.463888,Relatively Moderate
313,NaN,Connecticut,Northeastern Connecticut Planning Region,NaN,1.178378e+07,69.519567,Relatively Low
314,NaN,Connecticut,Northwest Hills Planning Region,NaN,2.323207e+07,82.500795,Relatively Low
315,NaN,Connecticut,South Central Connecticut Planning Region,NaN,8.602981e+07,94.463888,Relatively Moderate
316,NaN,Connecticut,Southeastern Connecticut Planning Region,NaN,3.323352e+07,87.368756,Relatively Moderate
317,NaN,Connecticut,Western Planning Region,NaN,1.045320e+07,95.450207,Relatively High


In [15]:
#THIS ONLY RUNS PROPERLY WHEN THE NRI DATASET IS FIXED IN THE PREVIOUS CELL
#this is because .compare() requires the length of the datasets to be the same, but originally, demo had 1 more row than nri from the connecticut problem

#use differences to see how each dataset named counties differently (basically same name but different format for most)
differences = demo["COUNTY"].compare(nri["COUNTY"])
differences

#simple differences in names are alaska, lousiana, virginia
#differences are alaska + connecticut, which set delaware, dc, florida, etc off balance

,self,other
67,Aleutians East Borough,Aleutians East
68,Aleutians West Census Area,Aleutians West
69,Anchorage Municipality,Anchorage
70,Bethel Census Area,Bethel
71,Bristol Bay Borough,Bristol Bay
...,...,...
2950,Suffolk city,Suffolk
2951,Virginia Beach city,Virginia Beach
2952,Waynesboro city,Waynesboro
2953,Williamsburg city,Williamsburg


In [16]:
#merging both dataframes so we can work with 1 big one
df = pd.merge(demo, nri, left_index=True, right_index=True)
df

,COUNTY_x,STATE_x,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,...,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,NRI_ID,STATE_y,COUNTY_y,POPULATION,RISK_VALUE,RISK_SCORE,RISK_RATNG
0,Autauga,Alabama,76,119518,58210,61308,44052,45342,11912,13606,...,44,44,38,C01001,Alabama,Autauga,58764.0,6.156054e+06,49.220490,Relatively Low
1,Baldwin,Alabama,76,492870,240328,252542,210450,220288,20202,21316,...,264,176,172,C01003,Alabama,Baldwin,231365.0,2.106327e+08,97.709195,Relatively High
2,Barbour,Alabama,76,49412,26128,23284,12846,11452,12434,11144,...,18,66,36,C01005,Alabama,Barbour,25160.0,7.412840e+06,56.188355,Relatively Low
3,Bibb,Alabama,76,44010,23550,20460,17274,16400,5728,3518,...,20,38,12,C01007,Alabama,Bibb,22239.0,3.863747e+06,32.484887,Very Low
4,Blount,Alabama,76,119024,59166,59858,56300,57236,1280,1030,...,58,74,56,C01009,Alabama,Blount,58992.0,1.023854e+07,65.128858,Relatively Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,Sweetwater,Wyoming,76,82690,42938,39752,40272,37364,646,448,...,86,44,44,C56037,Wyoming,Sweetwater,42238.0,1.189081e+06,6.617881,Very Low
3140,Teton,Wyoming,76,46574,24412,22162,23040,20832,220,172,...,44,26,26,C56039,Wyoming,Teton,23250.0,1.141156e+07,68.501432,Relatively Low
3141,Uinta,Wyoming,76,41424,21196,20228,20240,19240,146,126,...,32,12,14,C56041,Wyoming,Uinta,20412.0,2.821381e+06,21.699014,Very Low
3142,Washakie,Wyoming,76,15438,7946,7492,7510,7068,52,32,...,16,12,4,C56043,Wyoming,Washakie,7662.0,6.211963e+05,2.513522,Very Low


In [17]:
#rename the columns
df.rename(columns = {'WAC_MALE':'WHITE_M', 'WAC_FEMALE':'WHITE_F', 'BAC_MALE':'BLACK_M ', 'BAC_FEMALE':'BLACK_F', 'IAC_MALE':'NATIVE_M', 'IAC_FEMALE':'NATIVE_F', 'AAC_MALE':'ASIAN_M', 'AAC_FEMALE':'ASIAN_F', 'NAC_MALE':'HAW_PACIF_M', 'NAC_FEMALE':'HAW_PACIF_F', 'H_MALE':'HISP_M','H_FEMALE':'HISP_F'},inplace = True)

NEXT STEPS:

1. choose which columns to keep (county, state, total pop, demographics, risk value + score + rating)

1a. demographics to keep: use the "alone or in combination" to include more people (even though it might double count)

how we should rename = how it is named in demo

WHITE_M = WAC_MALE

WHITE_F = WAC_FEMALE

BLACK_M = BAC_MALE

BLACK_F = BAC_FEMALE

NATIVE_M = IAC_MALE

NATIVE_F = IAC_FEMALE

ASIAN_M = AAC_MALE

ASIAN_F = AAC_FEMALE

HAW_PACIF_M = NAC_MALE

HAW_PACIF_F = NAC_FEMALE

HISP_M = H_MALE

HISP_F = H_FEMALE

Also maybe keep TOT_MALE and TOT_FEMALE ??

2. rename column names to be more clear (finished)
3. add new columns and calculate percentages of each
4. possibly a basic visualization of this all

In [42]:
#calculate each race percentage per county
df['PERCENT_WHITE'] = (df['WA_FEMALE'] + df['WA_MALE']) / df['TOT_POP']
df['PERCENT_BLACK'] = (df['BA_FEMALE'] + df['BA_MALE']) / df['TOT_POP']
df['PERCENT_ASIAN'] = (df['AA_FEMALE'] + df['AA_MALE']) / df['TOT_POP']
df['PERCENT_NATIVE'] = (df['NA_FEMALE'] + df['NA_MALE']) / df['TOT_POP']
df['PERCENT_HISP'] = (df['HISP_F'] + df['HISP_M']) / df['TOT_POP']
df['PERCENT_HAW'] = (df['HAW_PACIF_F'] + df['HAW_PACIF_M']) / df['TOT_POP']

#white or non-white majority column
df['MAJORITY_RACE'] = df.apply(lambda _: '', axis=1)
df.loc[df['PERCENT_WHITE'] > 0.5, 'MAJORITY_RACE'] = 'WHITE'
df.loc[df['PERCENT_WHITE'] <= 0.5, 'MAJORITY_RACE'] = 'MINORITY'

df

,COUNTY_x,STATE_x,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,...,RISK_VALUE,RISK_SCORE,RISK_RATNG,PERCENT_WHITE,PERCENT_BLACK,PERCENT_ASIAN,PERCENT_NATIVE,PERCENT_HISP,PERCENT_HAW,MAJORITY_RACE
0,Autauga,Alabama,76,119518,58210,61308,44052,45342,11912,13606,...,6.156054e+06,49.220490,Relatively Low,0.747954,0.213508,0.011781,0.001222,0.033819,0.002192,WHITE
1,Baldwin,Alabama,76,492870,240328,252542,210450,220288,20202,21316,...,2.106327e+08,97.709195,Relatively High,0.873938,0.084237,0.011561,0.000665,0.050362,0.002414,WHITE
2,Barbour,Alabama,76,49412,26128,23284,12846,11452,12434,11144,...,7.412840e+06,56.188355,Relatively Low,0.491743,0.477172,0.004695,0.002793,0.051000,0.004048,MINORITY
3,Bibb,Alabama,76,44010,23550,20460,17274,16400,5728,3518,...,3.863747e+06,32.484887,Very Low,0.765144,0.210089,0.002908,0.001318,0.031766,0.001954,WHITE
4,Blount,Alabama,76,119024,59166,59858,56300,57236,1280,1030,...,1.023854e+07,65.128858,Relatively Low,0.953892,0.019408,0.004352,0.001311,0.100702,0.001949,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,Sweetwater,Wyoming,76,82690,42938,39752,40272,37364,646,448,...,1.189081e+06,6.617881,Very Low,0.938880,0.013230,0.009143,0.001621,0.166453,0.003048,WHITE
3140,Teton,Wyoming,76,46574,24412,22162,23040,20832,220,172,...,1.141156e+07,68.501432,Relatively Low,0.941985,0.008417,0.018809,0.001589,0.157642,0.003264,WHITE
3141,Uinta,Wyoming,76,41424,21196,20228,20240,19240,146,126,...,2.821381e+06,21.699014,Very Low,0.953071,0.006566,0.005939,0.001593,0.101053,0.003911,WHITE
3142,Washakie,Wyoming,76,15438,7946,7492,7510,7068,52,32,...,6.211963e+05,2.513522,Very Low,0.944293,0.005441,0.008680,0.000907,0.141728,0.002721,WHITE
